# Setting up data and env

## Upload Your Project

In [ ]:
from google.colab import files
print("Upload your project645.zip file...")
uploaded = files.upload()

# Verify upload
for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

Upload your project645.zip file...


Saving project645.zip to project645.zip
User uploaded file "project645.zip" with length 24124184 bytes


## Unzip the project

In [ ]:
!unzip -q project645.zip -d /content/
# Verify unzip by listing contents
!ls /content/project645/
!ls /content/project645/code/

code  LICENSE  mai645.yml  README.md  train_data_bvh
analysis.py			 read_bvh.py
fix_feet.py			 read_bvh.pyc
generate_training_euler_data.py  rotation2xyz.py
generate_training_pos_data.py	 rotation2xyz.pyc
generate_training_quad_data.py	 rotation_conversions.py
__pycache__			 synthesize_euler_motion.py
pytorch_train_euler_aclstm.py	 synthesize_pos_motion_original_colab.py
pytorch_train_pos_aclstm.py	 synthesize_pos_motion.py
pytorch_train_quad_aclstm.py	 synthesize_quad_motion_debug.py
read_bvh_hierarchy.py		 synthesize_quad_motion.py
read_bvh_hierarchy.pyc		 test_encodings.py


## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Create Output Folder

*   **Create an Output Folder in Google Drive:** It's highly recommended to create a dedicated folder in your Google Drive to store the outputs (trained models, generated data/BVH files). For example, you could create a folder named `mai645_project_outputs` directly in your "My Drive".
*   Define a variable in Colab pointing to this Drive folder. **Make sure the path matches the folder you created.**

## **** WARNING YOU NEED TO MANUALLY ADD THE PREPROCCESSED DATA FOR QUADS AND POS IN THE FOLDER mai645_project_outputs_analysis AND RUN ONLY THE LOADING DATA STEPS BELOW FOR THE PATHS TO WORK AS EXPECTED****

In [ ]:
# IMPORTANT: Adjust this path if you named your Drive folder differently!
GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/mai645_project_outputs_analysis"
# Create the directory in Drive if it doesn't exist (optional, Colab can create it)
import os
os.makedirs(GDRIVE_OUTPUT_DIR, exist_ok=True)
print(f"Outputs will be saved to: {GDRIVE_OUTPUT_DIR}")

Outputs will be saved to: /content/drive/MyDrive/mai645_project_outputs_analysis


## Install Dependencies

In [ ]:
# Install the latest compatible PyTorch, torchvision, torchaudio for this Colab environment
!pip install -q torch torchvision torchaudio

# Install other libraries, specifically targeting numpy in the 2.0.x range.
# This aims to satisfy thinc (>=2.0.0) AND tensorflow/numba (<2.1.0).
!pip install -q contourpy==1.3.1 cycler==0.12.1 fonttools==4.56.0 kiwisolver==1.4.8 matplotlib==3.10.1 "numpy>=2.0.0,<2.1.0" opencv-python==4.11.0.86 packaging==24.2 pyparsing==3.2.3 python-dateutil==2.9.0.post0 six==1.17.0 transforms3d==0.4.2

# Verify installation
!pip show torch torchvision torchaudio transforms3d numpy opencv-python tensorflow numba

## Change the current working directory to where your Python scripts are located

In [ ]:
cd /content/project645/code/

/content/project645/code


# Positional Preprocessing

## Upload preprocessed positional data

In [ ]:
import os
# Ensure GDRIVE_OUTPUT_DIR is defined (should match your Drive output folder)
# GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/mai645_project_outputs" # Uncomment and adjust if not run in a previous cell

# Define source directories in Google Drive for positional data
GDRIVE_POS_TRAIN_DATA_SOURCE_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "train_data_pos", "salsa")
GDRIVE_POS_RECON_BVH_SOURCE_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "reconstructed_bvh_data_pos", "salsa")

# Define target Colab directories
COLAB_POS_TRAIN_DATA_DIR = "/content/project645/train_data_pos/salsa"
COLAB_POS_RECON_BVH_DIR = "/content/project645/reconstructed_bvh_data_pos/salsa"

# Create target directories in Colab if they don't exist
os.makedirs(COLAB_POS_TRAIN_DATA_DIR, exist_ok=True)
os.makedirs(COLAB_POS_RECON_BVH_DIR, exist_ok=True)

print(f"Attempting to copy from {GDRIVE_POS_TRAIN_DATA_SOURCE_DIR} to {COLAB_POS_TRAIN_DATA_DIR}...")
if os.path.exists(GDRIVE_POS_TRAIN_DATA_SOURCE_DIR) and os.listdir(GDRIVE_POS_TRAIN_DATA_SOURCE_DIR):
  get_ipython().system(f"cp -r {GDRIVE_POS_TRAIN_DATA_SOURCE_DIR}/* {COLAB_POS_TRAIN_DATA_DIR}/")
  print(f"Copying training data from Drive complete. Check: !ls {COLAB_POS_TRAIN_DATA_DIR}")
else:
  print(f"Source directory {GDRIVE_POS_TRAIN_DATA_SOURCE_DIR} is empty or does not exist in Google Drive. Skipping copy.")

print(f"Attempting to copy from {GDRIVE_POS_RECON_BVH_SOURCE_DIR} to {COLAB_POS_RECON_BVH_DIR}...")
if os.path.exists(GDRIVE_POS_RECON_BVH_SOURCE_DIR) and os.listdir(GDRIVE_POS_RECON_BVH_SOURCE_DIR):
  get_ipython().system(f"cp -r {GDRIVE_POS_RECON_BVH_SOURCE_DIR}/* {COLAB_POS_RECON_BVH_DIR}/")
  print(f"Copying reconstructed BVH data from Drive complete. Check: !ls {COLAB_POS_RECON_BVH_DIR}")
else:
  print(f"Source directory {GDRIVE_POS_RECON_BVH_SOURCE_DIR} is empty or does not exist in Google Drive. Skipping copy.")

print("Process to load positional data from Google Drive finished.")
# You can verify by listing contents in Colab:
# !ls /content/project645/train_data_pos/salsa/
# !ls /content/project645/reconstructed_bvh_data_pos/salsa/

Attempting to copy from /content/drive/MyDrive/mai645_project_outputs_analysis/train_data_pos/salsa to /content/project645/train_data_pos/salsa...
Copying training data from Drive complete. Check: !ls /content/project645/train_data_pos/salsa
Attempting to copy from /content/drive/MyDrive/mai645_project_outputs_analysis/reconstructed_bvh_data_pos/salsa to /content/project645/reconstructed_bvh_data_pos/salsa...
Source directory /content/drive/MyDrive/mai645_project_outputs_analysis/reconstructed_bvh_data_pos/salsa is empty or does not exist in Google Drive. Skipping copy.
Process to load positional data from Google Drive finished.


## (19m) Preprocess Positional data

In [ ]:
# Preprocess Positional data
!python generate_training_pos_data.py
# Check if output folders were created for salsa
!ls /content/project645/train_data_pos/
!ls /content/project645/train_data_pos/salsa/
!ls /content/project645/reconstructed_bvh_data_pos/salsa/

Traceback (most recent call last):
object address  : 0x7b7d771fbdc0
object refcount : 2
object type     : 0x9d5ea0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C
salsa


## Save Positional Data to Drive & Download (Optional)

In [ ]:
# Ensure GDRIVE_OUTPUT_DIR is defined
# GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/mai645_project_outputs" # Uncomment and adjust if not run in a previous cell

# Define source and target directories for positional data
COLAB_POS_TRAIN_DATA_DIR = "/content/project645/train_data_pos/salsa"
COLAB_POS_RECON_BVH_DIR = "/content/project645/reconstructed_bvh_data_pos/salsa"

GDRIVE_POS_TRAIN_DATA_TARGET_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "train_data_pos", "salsa")
GDRIVE_POS_RECON_BVH_TARGET_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "reconstructed_bvh_data_pos", "salsa")

# Create target directories in Google Drive
os.makedirs(GDRIVE_POS_TRAIN_DATA_TARGET_DIR, exist_ok=True)
os.makedirs(GDRIVE_POS_RECON_BVH_TARGET_DIR, exist_ok=True)

print(f"Copying {COLAB_POS_TRAIN_DATA_DIR} to {GDRIVE_POS_TRAIN_DATA_TARGET_DIR}...")
# Use !cp for shell command execution in Colab
# Make sure the source directory is not empty before copying
# Adding a check here, or ensure your preprocessing script ran successfully
if os.path.exists(COLAB_POS_TRAIN_DATA_DIR) and os.listdir(COLAB_POS_TRAIN_DATA_DIR):
  get_ipython().system(f"cp -r {COLAB_POS_TRAIN_DATA_DIR}/* {GDRIVE_POS_TRAIN_DATA_TARGET_DIR}/")
  print("Copying training data complete.")
else:
  print(f"Source directory {COLAB_POS_TRAIN_DATA_DIR} is empty or does not exist. Skipping copy.")

print(f"Copying {COLAB_POS_RECON_BVH_DIR} to {GDRIVE_POS_RECON_BVH_TARGET_DIR}...")
if os.path.exists(COLAB_POS_RECON_BVH_DIR) and os.listdir(COLAB_POS_RECON_BVH_DIR):
  get_ipython().system(f"cp -r {COLAB_POS_RECON_BVH_DIR}/* {GDRIVE_POS_RECON_BVH_TARGET_DIR}/")
  print("Copying reconstructed BVH data complete.")
else:
  print(f"Source directory {COLAB_POS_RECON_BVH_DIR} is empty or does not exist. Skipping copy.")

# Zip the data for download
ZIP_POS_TRAIN_NAME = "project645_train_data_pos_salsa.zip"
ZIP_POS_RECON_NAME = "project645_reconstructed_bvh_pos_salsa.zip"

print(f"Zipping {COLAB_POS_TRAIN_DATA_DIR} to {ZIP_POS_TRAIN_NAME}...")
if os.path.exists(COLAB_POS_TRAIN_DATA_DIR) and os.listdir(COLAB_POS_TRAIN_DATA_DIR):
    get_ipython().system(f"zip -r -q /content/{ZIP_POS_TRAIN_NAME} {COLAB_POS_TRAIN_DATA_DIR}")
    print(f"Zipping training data complete. Ready for download: {ZIP_POS_TRAIN_NAME}")
    try:
        files.download(f"/content/{ZIP_POS_TRAIN_NAME}")
    except NameError: # files might not be imported if this cell is run standalone
        from google.colab import files
        files.download(f"/content/{ZIP_POS_TRAIN_NAME}")
else:
    print(f"Source directory {COLAB_POS_TRAIN_DATA_DIR} is empty or does not exist. Skipping zip and download.")


print(f"Zipping {COLAB_POS_RECON_BVH_DIR} to {ZIP_POS_RECON_NAME}...")
if os.path.exists(COLAB_POS_RECON_BVH_DIR) and os.listdir(COLAB_POS_RECON_BVH_DIR):
    get_ipython().system(f"zip -r -q /content/{ZIP_POS_RECON_NAME} {COLAB_POS_RECON_BVH_DIR}")
    print(f"Zipping reconstructed BVH data complete. Ready for download: {ZIP_POS_RECON_NAME}")
    try:
        files.download(f"/content/{ZIP_POS_RECON_NAME}")
    except NameError:
        from google.colab import files
        files.download(f"/content/{ZIP_POS_RECON_NAME}")
else:
    print(f"Source directory {COLAB_POS_RECON_BVH_DIR} is empty or does not exist. Skipping zip and download.")

print("Data saving and zipping process finished.")
# You can verify by listing contents in Drive:
# !ls "{GDRIVE_POS_TRAIN_DATA_TARGET_DIR}"
# !ls "{GDRIVE_POS_RECON_BVH_TARGET_DIR}"

Copying /content/project645/train_data_pos/salsa to /content/drive/MyDrive/mai645_project_outputs_analysis/train_data_pos/salsa...
Source directory /content/project645/train_data_pos/salsa is empty or does not exist. Skipping copy.
Copying /content/project645/reconstructed_bvh_data_pos/salsa to /content/drive/MyDrive/mai645_project_outputs_analysis/reconstructed_bvh_data_pos/salsa...
Source directory /content/project645/reconstructed_bvh_data_pos/salsa is empty or does not exist. Skipping copy.
Zipping /content/project645/train_data_pos/salsa to project645_train_data_pos_salsa.zip...
Source directory /content/project645/train_data_pos/salsa is empty or does not exist. Skipping zip and download.
Zipping /content/project645/reconstructed_bvh_data_pos/salsa to project645_reconstructed_bvh_pos_salsa.zip...
Source directory /content/project645/reconstructed_bvh_data_pos/salsa is empty or does not exist. Skipping zip and download.
Data saving and zipping process finished.


# Euler Preprocessing

In [ ]:
# Preprocess Euler data
!python generate_training_euler_data.py
# Check if output folders were created
!ls /content/project645/train_data_euler/
!ls /content/project645/train_data_euler/salsa/
!ls /content/project645/reconstructed_bvh_data_euler/salsa/

--- Euler Data Generation (with Hip Position Normalization) ---
Source BVH: ../train_data_bvh/salsa
Target .npy (Normalized Euler): ../train_data_euler/salsa/
Target reconstructed BVH: ../reconstructed_bvh_data_euler/salsa/
Standard BVH for reconstruction: ../train_data_bvh/standard.bvh
Hip Position Scale Factor: 0.01

Step 1: Converting BVH files to Normalized Euler representation (.npy)...
Created directory: ../train_data_euler/salsa/
Found 30 BVH files in ../train_data_bvh/salsa
Processing (encode): ../train_data_bvh/salsa/09.bvh
Saved normalized Euler data to ../train_data_euler/salsa/09.npy with shape (2576, 132)
Processing (encode): ../train_data_bvh/salsa/27.bvh
Saved normalized Euler data to ../train_data_euler/salsa/27.npy with shape (1691, 132)
Processing (encode): ../train_data_bvh/salsa/16.bvh
Saved normalized Euler data to ../train_data_euler/salsa/16.npy with shape (2243, 132)
Processing (encode): ../train_data_bvh/salsa/24.bvh
Saved normalized Euler data to ../train_data

# Quaternion Preprocess

In [ ]:
# Define path for cached preprocessed data archive on Google Drive
PREPROCESSED_QUAD_ARCHIVE_GDRIVE_PATH = os.path.join(GDRIVE_OUTPUT_DIR, "preprocessed_quad_data.tar.gz")
# Define a path to a key file that indicates preprocessing was successful and data is present
# This file is expected to be created by generate_training_quad_data.py
PREPROCESSED_QUAD_DONE_FLAG_PATH = "/content/project645/train_data_quad/salsa/metadata_quad.json"

# Variable to track if actual preprocessing is needed or if cached data can be used
preprocessing_needed = True

In [ ]:

# Load Cached Preprocessed Quaternion Data (if available)

if os.path.exists(PREPROCESSED_QUAD_ARCHIVE_GDRIVE_PATH):
  print(f"Found cached preprocessed data archive at {PREPROCESSED_QUAD_ARCHIVE_GDRIVE_PATH}.")
  print("Attempting to load...")
  # Ensure the base target directory for extraction exists
  !mkdir -p /content/project645
  # Copy from Drive to local Colab environment
  !cp "{PREPROCESSED_QUAD_ARCHIVE_GDRIVE_PATH}" /tmp/preprocessed_quad_data.tar.gz
  # Extract the archive into /content/project645/
  # The archive was created with paths relative to /content/project645
  print("Extracting data...")
  !tar -xzf /tmp/preprocessed_quad_data.tar.gz -C /content/project645/

  # Verify if a key file (flag path) now exists after extraction
  if os.path.exists(PREPROCESSED_QUAD_DONE_FLAG_PATH):
    print("Successfully loaded and verified cached preprocessed data.")
    preprocessing_needed = False
  else:
    print(f"Cached data archive found and extracted, but key file {PREPROCESSED_QUAD_DONE_FLAG_PATH} is missing.")
    print("This might indicate an issue with the cache or the extraction process. Will re-preprocess.")
    # Attempt to clean up potentially incomplete/corrupted extraction to avoid issues
    !rm -rf /content/project645/train_data_quad
    !rm -rf /content/project645/reconstructed_bvh_data_quad
    preprocessing_needed = True # Ensure it's true if verification fails
else:
  print(f"No cached preprocessed data archive found at {PREPROCESSED_QUAD_ARCHIVE_GDRIVE_PATH}.")
  print("Proceeding with fresh preprocessing.")
  preprocessing_needed = True # Explicitly set, though it's the default


Found cached preprocessed data archive at /content/drive/MyDrive/mai645_project_outputs_analysis/preprocessed_quad_data.tar.gz.
Attempting to load...
Extracting data...
Successfully loaded and verified cached preprocessed data.


In [ ]:
if preprocessing_needed:
  print("Running Quaternion preprocessing script as cached data was not used...")
  # Preprocess Quaternion data
  !python generate_training_quad_data.py

  # Verify that preprocessing script created the expected outputs and the flag file
  if os.path.exists(PREPROCESSED_QUAD_DONE_FLAG_PATH):
    print("Preprocessing script completed. Expected output flag file found.")
    print("Checking output directories:")
    !ls /content/project645/train_data_quad/
    !ls /content/project645/train_data_quad/salsa/
    !ls /content/project645/reconstructed_bvh_data_quad/salsa/

    # After successful preprocessing, save the results to Google Drive
    print("\nPreprocessing finished. Saving data to Google Drive for future runs...")
    # Create the archive.
    # -C /content/project645 makes paths in archive relative (e.g., train_data_quad, reconstructed_bvh_data_quad)
    # The directories to archive are /content/project645/train_data_quad and /content/project645/reconstructed_bvh_data_quad
    !tar -czf /tmp/preprocessed_quad_data.tar.gz -C /content/project645 train_data_quad reconstructed_bvh_data_quad
    !cp /tmp/preprocessed_quad_data.tar.gz "{PREPROCESSED_QUAD_ARCHIVE_GDRIVE_PATH}"
    print(f"Preprocessed data successfully saved to {PREPROCESSED_QUAD_ARCHIVE_GDRIVE_PATH}")
  else:
    print(f"Preprocessing script ran, but the key output flag file ({PREPROCESSED_QUAD_DONE_FLAG_PATH}) was not found.")
    print("Data will not be cached. Please check the preprocessing script and its outputs.")
    print("Listing contents of relevant directories for debugging (if they exist):")
    !ls -ld /content/project645/train_data_quad/ # Use -ld to show directory info or error
    !ls -ld /content/project645/train_data_quad/salsa/
    !ls -ld /content/project645/reconstructed_bvh_data_quad/
    !ls -ld /content/project645/reconstructed_bvh_data_quad/salsa/

else:
  print("Skipping Quaternion preprocessing as cached data was successfully loaded.")
  print("Verifying existence of loaded data directories:")
  # Confirm the directories expected from cache are present
  !ls /content/project645/train_data_quad/
  !ls /content/project645/train_data_quad/salsa/
  !ls /content/project645/reconstructed_bvh_data_quad/salsa/


Skipping Quaternion preprocessing as cached data was successfully loaded.
Verifying existence of loaded data directories:
salsa
01.npy	05.npy	09.npy	13.npy	17.npy	21.npy	25.npy	29.npy
02.npy	06.npy	10.npy	14.npy	18.npy	22.npy	26.npy	30.npy
03.npy	07.npy	11.npy	15.npy	19.npy	23.npy	27.npy	metadata_quad.json
04.npy	08.npy	12.npy	16.npy	20.npy	24.npy	28.npy
01.bvh	04.bvh	07.bvh	10.bvh	13.bvh	16.bvh	19.bvh	22.bvh	25.bvh	28.bvh
02.bvh	05.bvh	08.bvh	11.bvh	14.bvh	17.bvh	20.bvh	23.bvh	26.bvh	29.bvh
03.bvh	06.bvh	09.bvh	12.bvh	15.bvh	18.bvh	21.bvh	24.bvh	27.bvh	30.bvh


# Analysis

In [ ]:

# Ensure you are in the correct directory where analysis.py is located
# This should be /content/project645/code/
import os
# Verify current directory, should already be /content/project645/code/ from previous cells
print(f"Current directory for analysis: {os.getcwd()}")

# Before running analysis.py, make sure that:
# 1. The project645.zip has been uploaded and unzipped.
# 2. Google Drive is mounted.
# 3. The GDRIVE_OUTPUT_DIR variable is correctly set (e.g., mai645_project_outputs_con5 for Quad Con5 analysis).
#    The analysis.py script expects specific weight files to be present in paths constructed using this GDRIVE_OUTPUT_DIR.
#    For example, if GDRIVE_OUTPUT_DIR points to mai645_project_outputs_con5, then
#    QUAD_CON5_WEIGHT_PATH in analysis.py becomes "/content/drive/MyDrive/mai645_project_outputs_con5/weights_quad/0028000.weight".
#    Ensure these weight files (0049000.weight for Pos, 0051000.weight for Euler, 0028000.weight for Quad_Con5 etc.)
#    actually exist in your Google Drive output folders.
#    You might need to update the placeholder weight file names in analysis.py for QUAD_CON30 and QUAD_CON45.

# Also, ensure that the preprocessed data required by analysis.py (train_data_pos/salsa, etc.)
# exists in /content/project645/. This data is typically generated by the preprocessing steps
# earlier in this Colab notebook or loaded from a cache. If you skipped those, analysis.py might fail.

# List the contents of the code directory to confirm analysis.py is there
print("\nListing contents of /content/project645/code/:")
get_ipython().system('ls /content/project645/code/')

print("\nRunning analysis.py script...")
# The analysis.py script is expected to be in /content/project645/code/
# It will use relative paths for data (e.g., ../train_data_pos)
# and absolute paths for weights (e.g., /content/drive/MyDrive/mai645_project_outputs/weights_pos/...)
get_ipython().system('python analysis.py')

print("\n--- Analysis script execution finished. Check the output above for results and plots. ---")
print(f"Plots will be saved in /content/project645/results_quantitative_partB/ if the script ran correctly.")
print("You can also copy them to your Google Drive if needed, e.g.:")

Current directory for analysis: /content/project645/code

Listing contents of /content/project645/code/:
analysis.py			 read_bvh.pyc
fix_feet.py			 rotation2xyz.py
generate_training_euler_data.py  rotation2xyz.pyc
generate_training_pos_data.py	 rotation_conversions.py
generate_training_quad_data.py	 synthesize_euler_motion.py
__pycache__			 synthesize_pos_motion_original_colab.py
pytorch_train_euler_aclstm.py	 synthesize_pos_motion.py
pytorch_train_pos_aclstm.py	 synthesize_quad_motion_debug.py
pytorch_train_quad_aclstm.py	 synthesize_quad_motion.py
read_bvh_hierarchy.py		 temp_synthesis_output_partB
read_bvh_hierarchy.pyc		 test_encodings.py
read_bvh.py

Running analysis.py script...

--- Running Experiment: Positional ---
  Testing with initial_seq_len: 100
    Run 1/3 for seq_len 100...
      Run 1 metric: 0.0586
    Run 2/3 for seq_len 100...
      Run 2 metric: 0.0592
    Run 3/3 for seq_len 100...
      Run 3 metric: 0.0533
  Avg error for initial_seq_len 100: 0.0570 (+/- 0.0026)

# Plot loss positional and quad

to run this you need to use the file pytorch_train_pos_aclstm_for_calculating_loss_to_device by renaming it to pytorch_train_pos_aclstm. to handle a device error. but did not change the architecture or anything else. this is just to avoid changing the original file used for training.

In [ ]:
import os


# Directory to save the output plots in Colab environment
COLAB_PLOT_OUTPUT_DIR = "/content/project645/results_plot_loss"
os.makedirs(COLAB_PLOT_OUTPUT_DIR, exist_ok=True)

# Define paths to data (assuming preprocessing has been run)
POS_DATA_DIR = "/content/project645/train_data_pos/salsa/"
QUAD_DATA_DIR = "/content/project645/train_data_quad/salsa/"
METADATA_QUAD_PATH = os.path.join(QUAD_DATA_DIR, "metadata_quad.json")

# Define paths to weight folders.
# IMPORTANT: Ensure these directories exist in your Google Drive and contain the .weight files from respective training runs.
POS_WEIGHTS_DIR = "/content/drive/MyDrive/mai645_project_outputs/weights_pos"
QUAD_WEIGHTS_CON5_DIR = "/content/drive/MyDrive/mai645_project_outputs_con5/weights_quad"
QUAD_WEIGHTS_CON30_DIR = "/content/drive/MyDrive/mai645_project_outputs_con30/weights_quad"
QUAD_WEIGHTS_CON45_DIR = "/content/drive/MyDrive/mai645_project_outputs_con45/weights_quad"

# Check if the essential data and metadata paths exist before running
if not os.path.exists(POS_DATA_DIR):
    print(f"ERROR: POS data folder not found: {POS_DATA_DIR}")
    print("Please ensure POS preprocessing has been completed.")
if not os.path.exists(QUAD_DATA_DIR):
    print(f"ERROR: QUAD data folder not found: {QUAD_DATA_DIR}")
    print("Please ensure QUAD preprocessing has been completed.")
if not os.path.exists(METADATA_QUAD_PATH):
    print(f"ERROR: QUAD metadata file not found: {METADATA_QUAD_PATH}")
    print("Please ensure QUAD preprocessing has been completed.")

# Construct the command to run plot_loss.py
# Note: plot_loss.py is expected to be in /content/project645/code/
plot_script_path = "/content/project645/code/plot_loss.py"

# Output file paths for plots
pos_plot_output = os.path.join(COLAB_PLOT_OUTPUT_DIR, "pos_model_loss.png")
quad_plot_output_prefix = os.path.join(COLAB_PLOT_OUTPUT_DIR, "quad_model_loss") # _{config_label}.png will be appended

command = f"""
python {plot_script_path} \\
    --model_seq_len 100 \\
    --device cpu \\
    --pos_data_folder "{POS_DATA_DIR}" \\
    --pos_weights_folder "{POS_WEIGHTS_DIR}" \\
    --pos_output_plot "{pos_plot_output}" \\
    --quad_data_folder "{QUAD_DATA_DIR}" \\
    --quad_metadata_path "{METADATA_QUAD_PATH}" \\
    --quad_weights_folder_con5 "{QUAD_WEIGHTS_CON5_DIR}" \\
    --quad_weights_folder_con30 "{QUAD_WEIGHTS_CON30_DIR}" \\
    --quad_weights_folder_con45 "{QUAD_WEIGHTS_CON45_DIR}" \\
    --quad_output_plot_prefix "{quad_plot_output_prefix}"
"""

print("Running plot_loss.py script...")
print("This might take a few minutes depending on the number of weight files.")
print("\\nCommand being executed:")
print(command)

# Execute the command
get_ipython().system(command)

print(f"\\n--- Loss plotting script finished. ---")
print(f"Plots should be saved in: {COLAB_PLOT_OUTPUT_DIR}")
print("You can view them by navigating to the folder in the Colab file browser.")
print(f"To copy them to your Google Drive (e.g., into {GDRIVE_OUTPUT_DIR}/plots/):")
# print(f'!mkdir -p "{os.path.join(GDRIVE_OUTPUT_DIR, 'plots')}"')
# print(f'!cp -r {COLAB_PLOT_OUTPUT_DIR}/* "{os.path.join(GDRIVE_OUTPUT_DIR, 'plots/')}"')


Running plot_loss.py script...
This might take a few minutes depending on the number of weight files.
\nCommand being executed:

python /content/project645/code/plot_loss.py \
    --model_seq_len 100 \
    --device cpu \
    --pos_data_folder "/content/project645/train_data_pos/salsa/" \
    --pos_weights_folder "/content/drive/MyDrive/mai645_project_outputs/weights_pos" \
    --pos_output_plot "/content/project645/results_plot_loss/pos_model_loss.png" \
    --quad_data_folder "/content/project645/train_data_quad/salsa/" \
    --quad_metadata_path "/content/project645/train_data_quad/salsa/metadata_quad.json" \
    --quad_weights_folder_con5 "/content/drive/MyDrive/mai645_project_outputs_con5/weights_quad" \
    --quad_weights_folder_con30 "/content/drive/MyDrive/mai645_project_outputs_con30/weights_quad" \
    --quad_weights_folder_con45 "/content/drive/MyDrive/mai645_project_outputs_con45/weights_quad" \
    --quad_output_plot_prefix "/content/project645/results_plot_loss/quad_model_

# Plot loss euler

to run this you need to use the file pytorch_train_euler_aclstm_for_calculating_loss_to_device by renaming it to pytorch_train_euler_aclstm. to handle a device error. but did not change the architecture or anything else. this is just to avoid changing the original file used for training.

In [ ]:
import os

# Directory to save the output plots in Colab environment (should be already created)
# COLAB_PLOT_OUTPUT_DIR = "/content/project645/results_plot_loss"
# os.makedirs(COLAB_PLOT_OUTPUT_DIR, exist_ok=True) # Already created in previous cell

# Define paths to Euler data (assuming preprocessing has been run)
EULER_DATA_DIR = "/content/project645/train_data_euler/salsa/"

# Define path to Euler weight folder.
# IMPORTANT: Ensure this directory exists in your Google Drive and contains the .weight files from the Euler training run.
# This path should align with what was used in project645_2euler.py for training.
# Common structure might be GDRIVE_OUTPUT_DIR (e.g., /content/drive/MyDrive/mai645_project_outputs) + /weights_euler
# Or if you have a separate mai645_project_outputs_analysis, ensure weights_euler is there or point to the original training output dir.
# For this example, let's assume it's in the general GDRIVE_OUTPUT_DIR from earlier cells if not overridden for analysis specifically.
# If GDRIVE_OUTPUT_DIR = "/content/drive/MyDrive/mai645_project_outputs_analysis", then EULER_WEIGHTS_DIR might be:
# EULER_WEIGHTS_DIR = os.path.join(GDRIVE_OUTPUT_DIR, "weights_euler")
# OR, if weights are in the original output folder:
EULER_WEIGHTS_DIR = "/content/drive/MyDrive/mai645_project_outputs/weights_euler" # Adjust if your structure is different

# Check if the essential data and metadata paths exist before running
if not os.path.exists(EULER_DATA_DIR):
    print(f"ERROR: Euler data folder not found: {EULER_DATA_DIR}")
    print("Please ensure Euler preprocessing has been completed.")
if not os.path.exists(EULER_WEIGHTS_DIR):
    print(f"ERROR: Euler weights folder not found: {EULER_WEIGHTS_DIR}")
    print(f"Please ensure the folder exists and contains .weight files.")

# Construct the command to run plot_euler_loss.py
# Note: plot_euler_loss.py is expected to be in /content/project645/code/
euler_plot_script_path = "/content/project645/code/plot_euler_loss.py"

# Output file path for the Euler plot (in the same folder as other plots)
euler_plot_output = os.path.join(COLAB_PLOT_OUTPUT_DIR, "euler_model_loss.png")

# Frame size for Euler model is typically 132 as seen in project645_2euler.py
# It can also be auto-detected by the script if not provided.
euler_frame_channels = 132

euler_command = f"""
python {euler_plot_script_path} \
    --model_seq_len 100 \
    --device cpu \
    --euler_data_folder "{EULER_DATA_DIR}" \
    --euler_weights_folder "{EULER_WEIGHTS_DIR}" \
    --euler_output_plot "{euler_plot_output}" \
    --euler_in_frame_size {euler_frame_channels} # Optional, can be auto-detected
"""

print("Running plot_euler_loss.py script...")
print("This might take a few minutes depending on the number of weight files.")
print("\nCommand being executed for Euler loss plot:")
print(euler_command)

# Execute the command
if os.path.exists(EULER_DATA_DIR) and os.path.exists(EULER_WEIGHTS_DIR):
    get_ipython().system(euler_command)
    print(f"\n--- Euler loss plotting script finished. --- ")
    print(f"Plot should be saved in: {COLAB_PLOT_OUTPUT_DIR}")
else:
    print("Skipping Euler loss plot generation due to missing data or weights folder.")

# You can view them by navigating to the folder in the Colab file browser.
# To copy to Google Drive (if COLAB_PLOT_OUTPUT_DIR and GDRIVE_OUTPUT_DIR are set):
# print(f'!mkdir -p "{os.path.join(GDRIVE_OUTPUT_DIR, "plots")}' ) # Ensure plot dir exists on Drive
# print(f'!cp "{euler_plot_output}" "{os.path.join(GDRIVE_OUTPUT_DIR, "plots/")}"' ) if os.path.exists(euler_plot_output) else print("Euler plot not generated, skipping copy.")

Running plot_euler_loss.py script...
This might take a few minutes depending on the number of weight files.

Command being executed for Euler loss plot:

python /content/project645/code/plot_euler_loss.py     --model_seq_len 100     --device cpu     --euler_data_folder "/content/project645/train_data_euler/salsa/"     --euler_weights_folder "/content/drive/MyDrive/mai645_project_outputs/weights_euler"     --euler_output_plot "/content/project645/results_plot_loss/euler_model_loss.png"     --euler_in_frame_size 132 # Optional, can be auto-detected

Plotting Euler model loss from weights in: /content/drive/MyDrive/mai645_project_outputs/weights_euler
Loading motion files from /content/project645/train_data_euler/salsa/...
30 motion files loaded.
Using provided Euler in_frame_size: 132
Processing Euler weight: 0001000.weight (Iteration: 1000)
Processing Euler weight: 0002000.weight (Iteration: 2000)
Processing Euler weight: 0003000.weight (Iteration: 3000)
Processing Euler weight: 0004000